In [2]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import pickle
from utils import flat, trace
from tqdm import tqdm
import psutil
import torch
import gc
from itertools import product
from multiprocessing import Pool
from tqdm import tqdm
from data import RFCXDataset
from model import kaggle_metric
from utils import sigmoid

In [3]:
data_dir = '../../input/kaggle/train'
fnames = os.listdir(data_dir)
recording_ids = [x[:-5] for x in fnames]
print(len(fnames))
train_tp = pd.read_csv('../../input/kaggle/train_tp.csv', index_col='recording_id')
train_fp = pd.read_csv('../../input/kaggle/train_fp.csv', index_col='recording_id')
sample_submission = pd.read_csv('../../input/kaggle/sample_submission.csv')
folds = joblib.load('folds.jl')
print(train_tp.shape)
display(train_tp.head())
print(train_fp.shape)
display(train_fp.head())

4729
(1216, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,14,1,44.5440,2531.250,45.1307,5531.25
006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
009b760e6,10,1,50.0854,947.461,52.5293,10852.70


(7781, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
00204008d,21,1,13.8400,3281.2500,14.9333,4125.0000
00204008d,8,1,24.4960,3750.0000,28.6187,5531.2500
00204008d,4,1,15.0027,2343.7500,16.8587,4218.7500
003b04435,22,1,43.2533,10687.5000,44.8587,13687.5000
003b04435,23,1,9.1254,7235.1562,15.2091,11283.3984


In [10]:
sub = pd.read_csv('submission.csv', index_col='recording_id')
display(sub)
display(sub.describe())

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
recording_id,,,,,,,,,,,,,,,,,,,,,,,,
000316da7,0.008470,0.001382,0.000748,0.134327,0.000791,0.004586,0.000384,0.001060,0.001850,0.014293,0.005686,0.000179,0.009492,0.009189,0.002402,0.009933,0.009815,0.001278,0.121753,0.002374,0.007320,0.010515,0.013047,0.008646
003bc2cb2,0.002225,0.001957,0.000427,0.094559,0.000466,0.002973,0.003981,0.001547,0.012785,0.001971,0.015906,0.004061,0.000829,0.013694,0.007327,0.007448,0.646767,0.017078,0.007075,0.002264,0.019191,0.007191,0.016504,0.005978
0061c037e,0.006860,0.001540,0.002666,0.006966,0.002523,0.004343,0.002181,0.003574,0.006384,0.005126,0.692991,0.003306,0.003724,0.005080,0.002440,0.010479,0.013158,0.316934,0.006601,0.010746,0.018105,0.006265,0.002964,0.018367
010eb14d3,0.494639,0.004454,0.000608,0.247076,0.002923,0.006484,0.001143,0.019502,0.542334,0.016440,0.004157,0.002254,0.059400,0.031219,0.006041,0.001004,0.032716,0.002569,0.170838,0.003214,0.039431,0.007161,0.007217,0.011140
011318064,0.002802,0.001040,0.001533,0.125557,0.002209,0.001716,0.007358,0.002321,0.003433,0.002417,0.002710,0.004893,0.003074,0.002193,0.310273,0.232944,0.004752,0.005548,0.093081,0.003087,0.009831,0.015033,0.016019,0.010070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff68f3ac3,0.010508,0.092216,0.008560,0.188712,0.009312,0.540571,0.000818,0.021318,0.024402,0.000998,0.016288,0.000785,0.002217,0.006003,0.002771,0.629178,0.005576,0.004820,0.101812,0.008858,0.009991,0.007874,0.003346,0.944879
ff973e852,0.006991,0.001321,0.000632,0.009840,0.001360,0.003599,0.002091,0.317855,0.000692,0.152298,0.010599,0.002634,0.002300,0.002372,0.002328,0.085713,0.006382,0.040439,0.005912,0.016386,0.004080,0.002337,0.004369,0.008706
ffa5cf6d6,0.011056,0.066479,0.003155,0.115115,0.000672,0.004293,0.000757,0.000680,0.000801,0.033105,0.011142,0.000630,0.001200,0.004413,0.001416,0.505761,0.190348,0.072316,0.002298,0.007350,0.004487,0.004585,0.005702,0.007221


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
count,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000
mean,0.033364,0.047551,0.031277,0.106221,0.012516,0.039046,0.008055,0.071042,0.032586,0.028153,0.032832,0.045598,0.075687,0.032363,0.043307,0.063202,0.062135,0.023110,0.067778,0.013891,0.015975,0.022049,0.033418,0.086437
std,0.094517,0.119986,0.142224,0.073195,0.074034,0.129834,0.054584,0.143865,0.144120,0.090067,0.126007,0.143449,0.170803,0.095545,0.136110,0.139466,0.179177,0.107745,0.115110,0.068854,0.059544,0.093203,0.128082,0.205226
min,0.000650,0.000187,0.000094,0.002245,0.000186,0.000640,0.000097,0.000266,0.000114,0.000590,0.000468,0.000110,0.000374,0.000587,0.000254,0.000866,0.001540,0.000392,0.000580,0.000301,0.000822,0.001194,0.000818,0.002034
25%,0.002750,0.001441,0.000829,0.046448,0.000936,0.003254,0.000782,0.001382,0.001270,0.003930,0.003432,0.000806,0.001896,0.005082,0.001521,0.004485,0.007334,0.002012,0.004323,0.002665,0.004352,0.004581,0.005609,0.008824
50%,0.005289,0.003362,0.001732,0.097752,0.001633,0.005882,0.001617,0.004746,0.002465,0.007561,0.006404,0.001816,0.003682,0.010112,0.003221,0.009642,0.012693,0.003540,0.014347,0.004765,0.007065,0.007309,0.009072,0.016393
75%,0.012086,0.014566,0.003855,0.151572,0.003493,0.012030,0.003441,0.047037,0.005566,0.016277,0.012904,0.006249,0.012129,0.023381,0.008361,0.030430,0.024000,0.007169,0.085496,0.009042,0.012356,0.013486,0.014675,0.039844
max,0.751445,0.786906,0.983732,0.435597,0.956985,0.934562,0.946515,0.779191,0.971111,0.960209,0.983029,0.883436,0.799055,0.990366,0.899302,0.816754,0.980853,0.996028,0.951595,0.998630,0.987044,0.986724,0.997347,0.990608


In [11]:
sub.mean()

s0     0.033364
s1     0.047551
s2     0.031277
s3     0.106221
s4     0.012516
s5     0.039046
s6     0.008055
s7     0.071042
s8     0.032586
s9     0.028153
s10    0.032832
s11    0.045598
s12    0.075687
s13    0.032363
s14    0.043307
s15    0.063202
s16    0.062135
s17    0.023110
s18    0.067778
s19    0.013891
s20    0.015975
s21    0.022049
s22    0.033418
s23    0.086437
dtype: float64

In [7]:
train_tp

,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,14,1,44.5440,2531.250,45.1307,5531.25
006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
009b760e6,10,1,50.0854,947.461,52.5293,10852.70
...,...,...,...,...,...,...
fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75
fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25
ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00
